На основании представленных ссылок на спецификации различных моделей тракторов сделайте нейро-консультанта для ответов на вопросы покупателя, используя динамический выбор из разных баз знаний.

Ссылки для создания БЗ:

https://assets.cnhindustrial.com/nhag/apac/ru-ru/assets/pdf/agriculture-tractors/IR1106NCIS.pdf


http://agromester.md/pdf/9r.pdf

Проверьте работу нейро-консультанта на самостоятельно сформулированных вопросах

In [1]:
!pip install -q unstructured_inference pikepdf pypdf==4.0.2 unstructured pdf2image==1.17.0 pdfminer.six==20231228 faiss-cpu pillow_heif langchain langchain_community langchain_mistralai nltk sentence_transformers mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
# запустите эту ячейку, если используете секретный ключ в колабе
from sentence_transformers import SentenceTransformer
import os
from mistralai import Mistral
from google.colab import userdata
api_key = userdata.get("MISTRAL_API_KEY")
os.environ["MISTRAL_API_KEY"] = api_key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = Mistral(api_key)

In [ ]:
# запустите эту ячейку, если используете секретный environment
from sentence_transformers import SentenceTransformer
import os
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()  # Загружает переменные из .env файла
api_key = os.environ["MISTRAL_API_KEY"]

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = Mistral(api_key)

In [3]:
#@title Функция скачивает pdf-файл и сохраняет его

# импорт модулей
import requests

# функция принимает URL PDF-файла и путь, по которому файл будет сохранен в файловой системе Colab
# она скачивает файл и сохраняет его, если HTTP-запрос успешен
def download_pdf_from_url(url, save_path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Failed to download PDF. HTTP Status Code: {response.status_code}")

In [4]:
#@title Загрузка broshura_corolla.pdf
url = "https://assets.cnhindustrial.com/nhag/apac/ru-ru/assets/pdf/agriculture-tractors/IR1106NCIS.pdf"
save_path = "/content/IR1106NCIS.pdf"
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/IR1106NCIS.pdf


In [5]:
#@title Загрузка 9r.pdf
url = "http://agromester.md/pdf/9r.pdf"
save_path = "/content/9r.pdf"
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/9r.pdf


In [6]:
#@title Парсинг PDF

# установка библиотек
!pip install PyMuPDF==1.25.1  # работа с документами в формате PDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 26.7 MB/s eta 0:00:00


In [7]:
import numpy as np
import faiss

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Функция для создания эмбеддингов из массива документов
def create_embeddings(documents):
    return model.encode(documents)

def create_database(documents):
  # Создание эмбеддингов
  embeddings = create_embeddings(documents)

  # Создадим индексную базу из разделенных фрагментов текста
  # Преобразование эмбеддингов в массив numpy
  embeddings = np.array(embeddings).astype('float32')

  # Создание индекса FAISS
  dimension = embeddings.shape[1]  # Размерность векторов
  db = faiss.IndexFlatL2(dimension)  # Использование L2 метрики

  # Добавление векторов в индекс
  db.add(embeddings)
  return db, embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# импорт модулей
import fitz # работа с документами в формате PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter      # рекурсивное разделение текста

# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_db(pdf_names, pdf_folder_path):
    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=len)

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder_path, pdf_file)

        # Извлечение текста из PDF
        try:
            with fitz.open(pdf_path) as pdf:
                text = ""
                for page in pdf:
                    text += page.get_text()
        except Exception as e:
            print(f"Ошибка при обработке файла {pdf_file}: {e}")
            continue

        # Разделение текста на части
        documents = text_splitter.split_text(text)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    db, embeddings = create_database(all_documents)

    return db, embeddings, all_documents

In [10]:
#@title Создадим базу знаний по тракторам на основе спецификаций в формате pdf
pdf_names = ['IR1106NCIS.pdf']
pdf_folder_path = "./"#"./content"
db_ir, embeddings, documents_ir = create_faiss_db(pdf_names, pdf_folder_path)

In [11]:
#@title Создадим базу знаний по тракторам на основе спецификаций в формате pdf
pdf_names = ['9r.pdf']
pdf_folder_path = "./" #"./content"
db_9r, embeddings, documents_9r = create_faiss_db(pdf_names, pdf_folder_path)

In [12]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

def getRetriever(documents):
    model_name = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}
    embeddings = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs
            )
    vector_store = FAISS.from_texts(documents, embeddings)
    return vector_store.as_retriever()

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embed_model = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs
            )

<ipython-input-13-c499e5bdcaa3>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
documents_9r

['Тракторы серии 9R/RT\nОт 302 до 412 кВт (от 410 до 560 л.с.) 97/68EC \n2\n|\nОсвободите зверя\nПредставьте трактор мощностью до 560 л.с. Высокотехнологичный инновационный трактор, \n\xadготовый выполнить тяжелейшие сельскохозяйственные операции с легкостью.\nПознакомьтесь с новым трактором John\xa0Deere серии 9R, который способен ошеломить вас своей \nмощностью.\nПредлагаются четыре колесные и три гусеничные модели – 9410R, 9460R/9460RT, 9510R/9510RT \nили 9560R/9560RT – с двигателями \xadноминальной мощности от 410 л.с. до 560 л.с. и максимальной \nмощности от 451 до to 616 л.с.\nНо эти тракторы отличаются не только мощностью, но и дополнительными характеристиками. \n\xadТакими как комфортабельная, просторная кабина CommandView II и современные интегрирован\xad',
 '\xadТакими как комфортабельная, просторная кабина CommandView II и современные интегрирован\xad\nные функции систем точного земледелия AMS. Эти тракторы являются разумной инвестицией при \nпроведении широкомасштабных сель

In [15]:
retriever_9r = getRetriever(documents_9r)
retriever_ir = getRetriever(documents_ir)

In [16]:
#@title Добавляем информацию и описание для наших векторных хранилищ (баз знаний)

from langchain.prompts import PromptTemplate              # шаблона запроса
MULTI_RETRIEVAL_ROUTER_TEMPLATE = """
Given the user query, select the most appropriate retriever from the list below:
{descriptions}

Query: {query}
Selected Retriever:
"""
prompt_mult = PromptTemplate.from_template(MULTI_RETRIEVAL_ROUTER_TEMPLATE)

retriever_infos = [
    {
        "name": "JohnDeere9R/RT",
        "description": "Suitable for answering questions about models of the John Deere 9R/RT tractors",  # описываем, для каких вопросов применять данную БЗ
        "retriever": retriever_9r
    },
    {
        "name": "NEWHOLLANDT6OOO",
        "description": "Suitable for answering questions about models of the NEW HOLLAND T6OOO tractors",
        "retriever": retriever_ir
    }
]

In [135]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI               # класс ChatMistralAI
from langchain.chains.llm import LLMChain

# Промпт для определения модели трактора
system_prompt_extractor = """
Извлеки серию трактора из текста. Определи, о какой серии идет речь, даже если она указана косвенно.
Серия должна быть одной из следующих:
- John Deere 9R/RT
- NEW HOLLAND T6OOO

Для определения серии используй следующие признаки:
- Для "John Deere 9R/RT": упоминание о крупных тракторах John Deere или серии 9R/RT.
- Для "NEW HOLLAND T6OOO": упоминание о тракторах New Holland или серии T6OOO или если упоминается тракторы серии Т6000.

В ответе явно укажи серию трактора и в ответе верни только одно значение из следующих:
- John Deere 9R/RT
- NEW HOLLAND T6OOO
- UNKNOWN (если серию не удалось определить).

Ответ верни в формате json, где серию укажи в json поле seria.
В ответе ожидается только json.
Действуй так, как будто в контексте подразумевается только одна из указанных моделей.
Не отвечай на вопрос. Не добавляй пояснений. Не добавляй какого-либо текста, кроме названия серию.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_extractor),
        ("human", "{query}"),
    ]
)

llm = ChatMistralAI()

# Цепочка для обработки запроса
model_extractor = {"query": RunnablePassthrough()} | prompt | llm

In [151]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import json

# Предположим, что у вас есть словарь с ретриверами для каждой модели трактора
retrievers = {
    "John Deere 9R/RT": retriever_9r,
    "NEW HOLLAND T6OOO": retriever_ir,
    "UNKNOWN": RunnableLambda(lambda query: "Серия трактора не распознана, укажите явно серию John Deere 9R/RT или NEW HOLLAND T6OOO."),
}

def extract_seria(query):
  content = model_extractor.invoke(query).content
  json_pattern = r'\{.*\}'
  match = re.search(json_pattern, content, re.DOTALL)
  json_string = match.group(0)
  return json.loads(json_string).get("seria")

# Создаем цепочку с использованием LCEL
multiple_chain = (
    RunnablePassthrough()  # Передает запрос дальше
    | RunnableLambda(lambda query: {"model": extract_seria(query), "query": query})  # Извлекает модель и сохраняет запрос
    | RunnableLambda(lambda data: retrievers.get(data["model"], retrievers["UNKNOWN"]).invoke(data["query"]))  # Вызывает ретривер
)

In [144]:
#@title Формируем system

system = """
  Очень подробно и детально ответь на вопрос пользователя,
  опираясь точно на документ с информацией для ответа клиенту.
  Не упоминай документ.
  Не придумывай ничего от себя. Не ссылайся на сами отрывки документа
  с информацией для ответа, клиент о них ничего не должен знать.
  В ответе используй русский язык.
"""


In [152]:
import re
def multiple_chain_answer(system, question, verbose=1):
    #indices_list = indices[0].tolist()
    # Вывод ближайших документов
    docs = multiple_chain.invoke(question)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = Mistral(api_key=api_key)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{question}"}
    ]

    if verbose: print('\n ===========================================: ')
    completion = client.chat.complete(
        model="mistral-small-latest",             # выберите модель самостоятельно
        messages=messages,
        temperature=0          # укажите значение
    )
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [92]:
query = "Какие технологии автоматизации есть у тракторов серии John Deere 9R/RT?"
result = multiple_chain_answer(system, query, 0)
print(result)

Тракторы серии John Deere 9R/RT оснащены различными технологиями автоматизации, которые значительно повышают их производительность и удобство использования. Одной из ключевых технологий является система точного земледелия AMS, которая обеспечивает точную навигацию и управление сельскохозяйственными операциями. Эта система позволяет тракторам выполнять задачи с высокой точностью, что особенно важно для таких операций, как посев, обработка почвы и сбор урожая.

Кроме того, тракторы серии 9R/RT оснащены современными системами освещения на 360°, что обеспечивает отличную видимость в любых условиях, включая ночное время. Это позволяет операторам работать безопасно и эффективно в любое время суток.

Также стоит отметить, что тракторы этой серии оснащены мощными двигателями PowerTech объемом 13,5 литров и продвинутыми трансмиссиями, которые обеспечивают высокую мощность и эффективность. Эти компоненты работают в сочетании с гидравлическими системами и навесными устройствами, что позволяет тра

In [121]:
query = "Какие преимущества предоставляет система ЕРМ в тракторах серии Т6000, и как она влияет на мощность двигателя и расход топлива?"
result = multiple_chain_answer(system, query, 0)
print(result)

Система интеллектуального управления выходной мощностью (ЕРМ) в тракторах серии Т6000 предоставляет несколько ключевых преимуществ. Во-первых, она позволяет увеличить мощность двигателя при необходимости, что особенно полезно при выполнении энергоемких операций. Например, при работе с тяжелыми орудиями или при перевозке грузов система может автоматически повысить мощность двигателя, обеспечивая необходимую производительность.

Во-вторых, ЕРМ помогает оптимизировать расход топлива. Благодаря интеллектуальному управлению, система обеспечивает, что мощность двигателя используется только тогда, когда это действительно необходимо. Это означает, что в менее энергоемких операциях трактор будет работать на более низких оборотах, что снижает расход топлива и экономит ресурсы.

Таким образом, система ЕРМ в тракторах серии Т6000 обеспечивает высокую производительность при минимальных затратах топлива, делая работу трактора более эффективной и экономичной.


In [156]:
query = "Расскажи об особенностях системы навигации тракторов серии NEW HOLAND Т6OOO?"
result = multiple_chain_answer(system, query, 0)
print(result)

Тракторы серии New Holland T6000 оснащены инновационной системой навигации, которая обеспечивает высокую точность и удобство в управлении. Система навигации включает в себя несколько ключевых функций, которые делают работу с трактором максимально эффективной и удобной.

Во-первых, органы управления и настройки ВОМа (вал отбора мощности) расположены на панели модуля SideWinder II. Это позволяет оператору легко и быстро настраивать параметры работы ВОМа, не отвлекаясь от основных задач. Все настройки могут быть сохранены в памяти монитора IntelliView III, что обеспечивает удобство и повторяемость настроек.

Во-вторых, система автоматизированного прохождения поворотной полосы HTS (Headland Turn System) позволяет записывать и сохранять заданную последовательность действий при прохождении поворотной полосы. Это особенно полезно для снижения утомляемости оператора, так как система автоматически воспроизводит записанный алгоритм при активизации. Алгоритм HTS был разработан и совершенствован в

In [155]:
query = "Расскажи об особенностях системы навигации тракторов серии John Deere 9R/RT?"
result = multiple_chain_answer(system, query, 0)
print(result)

Тракторы серии John Deere 9R/RT оснащены системами точного земледелия AMS, которые обеспечивают полное освещение на 360°. Эти системы позволяют операторам тракторов выполнять сельскохозяйственные операции с высокой точностью и эффективностью. Системы точного земледелия AMS включают в себя современные технологии, которые помогают оптимизировать использование ресурсов, таких как топливо и удобрения, а также минимизировать воздействие на окружающую среду.
